In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib

#DATA CLEANING
matplotlib.rcParams["figure.figsize"] = (20, 20)

df1 = pd.read_csv("Bengaluru_House_Data.csv")
df1.head()

df1.groupby('area_type')['area_type'].agg('count')

df2 = df1.drop(['area_type', 'society', 'balcony', 'availability'], axis = 'columns')
df2.head()

df3 = df2.dropna()
df3.isnull().sum()

df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))

df3 = df2.dropna()
df3.isnull().sum()
df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))

def convert_sqft_to_nm(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    # If a no is like 1220
    try:
        return float(x)
    except:
        return None

df4=df3.copy()
df4['total_sqft'] = df4['total_sqft'].apply(convert_sqft_to_nm)

#FEATURE ENGINNERING & DIMENTIONALITY REDUCTION TECHNIQUES
df4.head(3)

df5=df4.copy()

# new feature
df5['price_per_sqrft'] = df5['price']*100000/df5['total_sqft']
# len(df5.location.unique())
df5.location=df5.location.apply(lambda x: x.strip())

location_stats = df5.groupby('location')['location'].agg('count').sort_values(ascending=False)

location_stats_less_than_10 =location_stats[location_stats<=10]
location_stats_less_than_10

df5.location = df5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
# Eliminating those locations which has location stats less than 10
len(df5.location.unique())


#OUTLIER REMOVAL
# Here we remove values that has extreme data values in them. One of the methods to detect and remove them is by standard deviation.

#Removing anomilies bc its unusual to have eg:- 1020 sqft home with 8 bhk :(
df6 = df5[~(df5.total_sqft/df5.bhk<300)]
# df6.shape


def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean (subdf.price_per_sqrft)
        st = np.std (subdf.price_per_sqrft)
        reduced_df = subdf[(subdf.price_per_sqrft>(m-st)) & (subdf.price_per_sqrft<=(m+st))]
        df_out = pd.concat([df_out, reduced_df], ignore_index=True)
    return df_out
    
df7 = remove_pps_outliers(df6)
# df7.head


#Drawing scatter graph
def plot_scatter_chart (df, location):
    bhk2= df[(df.location ==location) & (df.bhk==2)] 
    bhk3= df[(df.location==location) & (df.bhk==3)] 
    matplotlib.rcParams['figure.figsize'] = (15,10)
    plt.scatter (bhk2.total_sqft, bhk2.price_per_sqrft, color='blue', label='2 BHK', s=50)
    plt.scatter (bhk3.total_sqft, bhk3.price_per_sqrft,marker='+', color='green', label='3 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price")
    plt.title(location)
    plt.legend()
    
# plot_scatter_chart(df7, "Hebbal")

def remove_bhk_outliers (df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'): 
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'): 
            bhk_stats[bhk] = {
                'mean': np.mean (bhk_df.price_per_sqrft),
                'std': np.std(bhk_df.price_per_sqrft), 
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats=bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df [bhk_df.price_per_sqrft < (stats['mean'])]. index.values) 
    return df.drop(exclude_indices, axis='index')
df8 = remove_bhk_outliers (df7)
df8.shape

# plot_scatter_chart(df8, "Hebbal")

matplotlib.rcParams["figure.figsize"] = (20, 10)
# plt.hist(df8.price_per_sqrft, rwidth=0.5)
# plt.xlabel("Price per sqr feet")
# plt.ylabel("Count")

# plt.hist(df8.bath, rwidth=0.5)
# plt.xlabel("Price per sqr feet")
# plt.ylabel("Count")

df9=df8[df8.bath<df8.bhk+2]

df10= df9.drop(['size', 'price_per_sqrft'], axis = 'columns')
# df10.head(3)



# MODEL BUILDING (HOT ONE ENCODING) USING PANDAS METHOD

#Assigning numerical values to location column (As its in string)

#making locations numerical
dummies = pd.get_dummies(df10.location)

#dropping other column
df11 = pd.concat([df10,dummies.drop('other', axis='columns')], axis='columns')

#Dropping location column bc it has string and wherever location of place will be needed it will be represented in the form of numbers.
df12 = df11.drop('location', axis = 'columns')

#dropping dependent varibale price ( as its the only dependent column )
X = df12.drop('price', axis = 'columns')
X.head()

C:\Users\Shambhavi\AppData\Local\Temp\ipykernel_20252\821033735.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))
C:\Users\Shambhavi\AppData\Local\Temp\ipykernel_20252\821033735.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))


,total_sqft,bath,bhk,1st Block Jayanagar,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Block Hbr Layout,5th Phase JP Nagar,6th Phase JP Nagar,...,Vijayanagar,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur
0,2850.0,4.0,4,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1630.0,3.0,3,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1875.0,2.0,3,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1200.0,2.0,3,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1235.0,2.0,2,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [5]:
y = df12.price
y.head()

0    428.0
1    194.0
2    235.0
3    130.0
4    148.0
Name: price, dtype: float64

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)
lr_clf = LinearRegression()
# Training the model
lr_clf.fit(X_train, y_train)
# Testing the score of the model
lr_clf.score(X_test, y_test)

#Score --> 84.522%

0.8452277697874374

In [7]:
#Cross Validation - Cross validation is a technique used in machine learning to evaluate the performance of a model on unseen data.
# Cross validation is a technique used in machine learning to evaluate the performance of a model on unseen data. It involves dividing the available data
# into multiple folds or subsets, using one of these folds as a validation set, and training the model on the remaining folds.

# In machine learning, we couldn’t fit the model on the training data and can’t say that the model will work accurately for the real data. 
# For this, we must assure that our model got the correct patterns from the data, and it is not getting up too much noise. For this purpose,
# we use the cross-validation technique. 

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

# ShuffleSplit will randomize the sample so that each fold gets equal distribution of data samples.
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state = 0)

cross_val_score(LinearRegression(), X, y, cv = cv)

# Price Prediction

def predict_price(location, sqft, bath, bhk):
    #makind an array x
    loc_index = np.where(X.columns == location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index>=0:
        x[loc_index] = 1
    return lr_clf.predict([x])[0]

# The above function give sthe index of desired column
# E.g:-

# np.where(X.columns == '2nd Phase Judicial Layout')[0][0]
# x.columns

#Predicting price

predict_price('1st Phase JP Nagar', 1000, 2, 3)
# x.columns

C:\Users\Shambhavi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


81.72616900780261

In [38]:
predict_price('1st Phase JP Nagar', 1000, 3, 3)

C:\Users\Shambhavi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


86.80519395230952

In [13]:
# These artifacts --> pickle file and json file will be used by python flask server.

#import a picle file and pass model classifier as an argument
import pickle
with open('banglore_home_price_model.pickle', 'wb') as f:
    pickle.dump(lr_clf, f)


#making all our columns start with lower case and dumping it into json file.
import json
columns = {
    'data_columns': [col.lower() for col in X. columns]
}
with open("columns.json", "w") as f:
    f.write(json.dumps (columns))